## Общее описание:

Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас . 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Необходимо проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения.

 

## Описание данных

- `date` – дата совершения события

- `event` - событие

- `gender` – пол пользователя

- `os_name` – платформа пользователя

- `city` – город пользователя

- `device_id` – идентификатор устройства пользователя

- `urm_source` – канал, с которого пришел пользователь



### План

**1. Подготовка к работе. Импорт необходимых библиотек и загрузка данных** 

**2. Изучение данных**
   
**3. Подсчет MAU в феврале**

**4. Подсчет кол-ва установок в январе**

**5. Проведение когортного анализа - деление пользователей по дню установки. Подсчет конверсии из установки в покупку в течение 7 дней**

**6. Подсчет наибольшего кол-ва пользователей, которые пришли с платного маркетингового канала**

**7. Выяснение того, на каком этапе воронки отваливается бОльшая часть клиентов**

**8. Поиск канала, с которого пришедшие пользователи показали самую низкую конверсию в первую покупку**

**9. Поиск канала, с которого пришедшие пользователи имеют медианный первый чек выше остальных**

**10. Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI**

<hr style="border:1px solid">

## 1. Импортируем библиотеки и загружаем данные

In [67]:
import pandas as pd
import numpy as np

In [68]:
df = pd.read_csv('https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/p_UjGG_Dv8CE5Q')

In [69]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


<hr style="border:1px solid">

## 2. Изучаем данные

In [4]:
# Смотрим, какие есть уникальные значеняи событий

df.event.unique()

array(['app_start', 'choose_item', 'purchase', 'search', 'tap_basket',
       'app_install', 'register'], dtype=object)

In [5]:
# Смотрим, где есть пропущенные значения в данных. 
# Пропуски в столбце purchase_sum объясняются тем, что если событие покупки отсутствует, в столбце ставится пропуск,
# так что оставляем пропуски.

df.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

In [6]:
# Проверяем данные на наличие строк-дубликатов

df.loc[df.duplicated()]

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source


In [7]:
# Смотрим на типы данных

df.dtypes

date             object
event            object
purchase_sum    float64
os_name          object
device_id         int64
gender           object
city             object
utm_source       object
dtype: object

In [8]:
# Приводим столбец date к типу данных datetime

df['date'] = pd.to_datetime(df['date'])

<hr style="border:1px solid">

## 3. Считаем MAU в феврале

In [12]:
# Фильтруем данные по условию 2 месяца и считаем кол-во уникальных юзеров в приложении за месяц

mau = df.loc[df['date'].dt.month == 2]['device_id'].nunique()

In [13]:
mau

75032

<hr style="border:1px solid">

## 4. Считаем кол-во установок в январе

In [14]:
# Фильтруем данные по событию установки приложения и выбираем 1 месяц года.

installs_january = df.loc[(df['event'] == "app_install") & (df['date'].dt.month == 1)].shape[0]

In [15]:
installs_january

80297

<hr style="border:1px solid">

## 5. Проводим когортный анализ - делим пользователей по дню установки.
## Считаем конверсию из установки в покупку в течение 7 дней
## Для какой когорты конверсия была наибольшей?

<b>
Когорты будут формироваться по дню установки приложения.
<br><br>
    Будем считать пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней</b>

In [16]:
# Оставляем в данных только нужные нам столбцы

cohort_df = df[['date','device_id','event']]

In [17]:
cohort_df

,date,device_id,event
0,2020-01-01,669460,app_start
1,2020-01-01,833621,app_start
2,2020-01-01,1579237,app_start
3,2020-01-01,1737182,app_start
4,2020-01-01,4029024,app_start
...,...,...,...
2747963,2020-03-31,2984778,register
2747964,2020-03-31,27301864,register
2747965,2020-03-31,1294285,register
2747966,2020-03-31,3010574,register


In [18]:
# Создаем отдельный df с данными об установках

installs = cohort_df.query('event == "app_install"')

In [19]:
installs

,date,device_id,event
2515061,2020-01-01,4921563,app_install
2515062,2020-01-01,1311583,app_install
2515063,2020-01-01,23006325,app_install
2515064,2020-01-01,5645652,app_install
2515065,2020-01-01,8798567,app_install
...,...,...,...
2669653,2020-03-31,19717968,app_install
2669654,2020-03-31,32966665,app_install
2669655,2020-03-31,6335964,app_install
2669656,2020-03-31,29155826,app_install


In [20]:
# Создаем отдельный df с данными о покупках

purchases = cohort_df.query('event == "purchase"')

In [21]:
purchases

,date,device_id,event
7691,2020-01-01,6081155,purchase
7692,2020-01-01,77448,purchase
7693,2020-01-01,7458607,purchase
7694,2020-01-01,3520947,purchase
7695,2020-01-01,1170605,purchase
...,...,...,...
2507469,2020-03-31,11864116,purchase
2507470,2020-03-31,3403969,purchase
2507471,2020-03-31,26120362,purchase
2507472,2020-03-31,22011351,purchase


In [22]:
# Объединяем данные об установках с покупками

final_cohorts = installs.merge(purchases, how='left', on='device_id', suffixes=('_install', '_purchase'))

In [23]:
final_cohorts

,date_install,device_id,event_install,date_purchase,event_purchase
0,2020-01-01,4921563,app_install,2020-01-10,purchase
1,2020-01-01,4921563,app_install,2020-01-19,purchase
2,2020-01-01,4921563,app_install,2020-02-05,purchase
3,2020-01-01,4921563,app_install,2020-02-11,purchase
4,2020-01-01,4921563,app_install,2020-03-15,purchase
...,...,...,...,...,...
212359,2020-03-31,19717968,app_install,NaT,NaN
212360,2020-03-31,32966665,app_install,NaT,NaN
212361,2020-03-31,6335964,app_install,NaT,NaN
212362,2020-03-31,29155826,app_install,NaT,NaN


In [24]:
# Нам нужно знать, какие пользователи купили что-то в течение 7 дней после дня установки приложения


final_cohorts['has purchased in 7 days'] = final_cohorts['date_purchase'] - final_cohorts['date_install'] <= pd.Timedelta('7 days') 

In [25]:
final_cohorts

,date_install,device_id,event_install,date_purchase,event_purchase,has purchased in 7 days
0,2020-01-01,4921563,app_install,2020-01-10,purchase,False
1,2020-01-01,4921563,app_install,2020-01-19,purchase,False
2,2020-01-01,4921563,app_install,2020-02-05,purchase,False
3,2020-01-01,4921563,app_install,2020-02-11,purchase,False
4,2020-01-01,4921563,app_install,2020-03-15,purchase,False
...,...,...,...,...,...,...
212359,2020-03-31,19717968,app_install,NaT,NaN,False
212360,2020-03-31,32966665,app_install,NaT,NaN,False
212361,2020-03-31,6335964,app_install,NaT,NaN,False
212362,2020-03-31,29155826,app_install,NaT,NaN,False


In [27]:
# Группируем данные по когортам (дню установки) и считаем кол-во юзеров в каждой

grouped_installs = final_cohorts.groupby('date_install') \
.agg({'device_id':'nunique'}) \
.rename(columns={'device_id':'installs'})

In [28]:
grouped_installs

,installs
date_install,
2020-01-01,3579
2020-01-02,3144
2020-01-03,2402
2020-01-04,1831
2020-01-05,1671
...,...
2020-03-27,1199
2020-03-28,1091
2020-03-29,1117


In [29]:
# Теперь группируем данные по когортам с условием и считаем кол-во
# тех пользователей, которые что-то купили в течение 7 дней с даты установки

grouped_purchased = final_cohorts.query('`has purchased in 7 days` == True') \
.groupby('date_install') \
.agg({'device_id':'nunique'}) \
.rename(columns={'device_id':'purchases'})

In [30]:
grouped_purchased

,purchases
date_install,
2020-01-01,1408
2020-01-02,1186
2020-01-03,834
2020-01-04,639
2020-01-05,587
...,...
2020-03-27,277
2020-03-28,209
2020-03-29,223


In [31]:
# Объединяем данные в единый df с ключом когорты (date_install)

commom_df = pd.concat([grouped_installs,grouped_purchased], axis=1)

In [32]:
commom_df

,installs,purchases
date_install,,
2020-01-01,3579,1408
2020-01-02,3144,1186
2020-01-03,2402,834
2020-01-04,1831,639
2020-01-05,1671,587
...,...,...
2020-03-27,1199,277
2020-03-28,1091,209
2020-03-29,1117,223


In [33]:
# Считаем конверсию - отношение доли покупок к установкам в каждой когорте

commom_df['conversion'] = (commom_df['purchases'] / commom_df['installs']).round(3)

In [34]:
# Сортируем значение конверсии по убыванию

commom_df.sort_values('conversion', ascending=False)

,installs,purchases,conversion
date_install,,,
2020-01-01,3579,1408,0.393
2020-01-09,1424,558,0.392
2020-01-15,4310,1650,0.383
2020-01-14,5173,1973,0.381
2020-01-02,3144,1186,0.377
...,...,...,...
2020-03-18,1171,235,0.201
2020-03-29,1117,223,0.200
2020-03-22,1261,251,0.199


In [35]:
commom_df.loc[['2020-01-01']]['conversion'].mul(100).round(1)

date_install
2020-01-01    39.3
Name: conversion, dtype: float64

<b>Итог: наибольшая конверсия была в когорте 2020-01-01</b>

<hr style="border:1px solid">

## 6. С какого платного маркетингового канала пришло больше всего новых пользователей? 

In [4]:
# Оставляем в df только данные о пользователях, которые пришли с платных каналов

channels = df[~(df['utm_source'] == "-")]

In [6]:
channels.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
6,2020-01-01,app_start,NaN,android,8383386,female,Saint-Petersburg,vk_ads


In [11]:
# Группируем данные по каналу и считаем кол-во пользователей

users_from_channels = channels.groupby('utm_source', as_index=False) \
.agg({'device_id':'nunique'}) \
.sort_values('device_id', ascending=False)

In [12]:
users_from_channels

,utm_source,device_id
5,yandex-direct,40712
1,google_ads,38096
4,vk_ads,34086
2,instagram_ads,31048
0,facebook_ads,25959
3,referal,15926


<b>Вывод: Больше всего пользователей пришло с Яндекса</b>

<hr style="border:1px solid">

## 7. На каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

### 7.1. Подготавливаем данные

In [13]:
# Создаем новый df с нудными нам столбцами

funnel = df.drop(['os_name', 'gender','city', 'utm_source'], axis=1)

In [14]:
funnel.head()

,date,event,purchase_sum,device_id
0,2020-01-01,app_start,NaN,669460
1,2020-01-01,app_start,NaN,833621
2,2020-01-01,app_start,NaN,1579237
3,2020-01-01,app_start,NaN,1737182
4,2020-01-01,app_start,NaN,4029024


In [15]:
# Для исследования этапов воронки отдельно для зарегистрированных и незарегистрированных пользователей,
# сначала разобьем данные на df с данными о дате регистрации того или иного пользователя и df с остальными действиями

funnel_actions = funnel.query('event != "register"')

In [16]:
funnel_actions.head()

,date,event,purchase_sum,device_id
0,2020-01-01,app_start,NaN,669460
1,2020-01-01,app_start,NaN,833621
2,2020-01-01,app_start,NaN,1579237
3,2020-01-01,app_start,NaN,1737182
4,2020-01-01,app_start,NaN,4029024


In [17]:
funnel_registering = funnel.query('event == "register"')[['date','device_id']]

In [18]:
funnel_registering.head()

,date,device_id
2669658,2020-01-01,294193
2669659,2020-01-01,22917617
2669660,2020-01-01,15248490
2669661,2020-01-01,252062
2669662,2020-01-01,2251583


In [19]:
# Объединяем данные

full_data = funnel_actions.merge(funnel_registering, how='left', on='device_id', suffixes=('_actions','_register'))

In [20]:
full_data.head()

,date_actions,event,purchase_sum,device_id,date_register
0,2020-01-01,app_start,NaN,669460,2020-01-07
1,2020-01-01,app_start,NaN,833621,2020-01-01
2,2020-01-01,app_start,NaN,1579237,2020-01-01
3,2020-01-01,app_start,NaN,1737182,2020-01-01
4,2020-01-01,app_start,NaN,4029024,NaN


In [21]:
# Узнаем, какие пользователи зарегистрированы по условию - позже ли дата события даты регистрации

full_data['is_registered'] = np.where(full_data['date_actions'] > full_data['date_register'], 'registered','not_registered')

In [22]:
full_data.head()

,date_actions,event,purchase_sum,device_id,date_register,is_registered
0,2020-01-01,app_start,NaN,669460,2020-01-07,not_registered
1,2020-01-01,app_start,NaN,833621,2020-01-01,not_registered
2,2020-01-01,app_start,NaN,1579237,2020-01-01,not_registered
3,2020-01-01,app_start,NaN,1737182,2020-01-01,not_registered
4,2020-01-01,app_start,NaN,4029024,NaN,not_registered


### 7.2 Смотрим относительную конверсию для зарегистрированных пользователей

In [23]:
# Фильтруем df по условию регистрации пользователя

registered_users = full_data.query('is_registered == "registered"')

In [24]:
registered_users.head()

,date_actions,event,purchase_sum,device_id,date_register,is_registered
15148,2020-01-02,app_start,NaN,3163406,2020-01-01,registered
15175,2020-01-02,app_start,NaN,2169108,2020-01-01,registered
15197,2020-01-02,app_start,NaN,4863305,2020-01-01,registered
15210,2020-01-02,app_start,NaN,15248490,2020-01-01,registered
15214,2020-01-02,app_start,NaN,25617659,2020-01-01,registered


In [47]:
# Группируем по событиям и считаем кол-во пользователей в каждом шаге

registered_users_conv = registered_users.groupby('event') \
.agg({'device_id':'nunique'}) \
.rename(columns={'device_id':'users'}) \
.reset_index()

In [48]:
registered_users_conv

,event,users
0,app_start,40991
1,choose_item,37926
2,purchase,24880
3,search,40482
4,tap_basket,34517


In [ ]:
# Для удобства форматируем df с помощью функции pivot


registered_users_conv.index = [0] * len(registered_users_conv)

In [51]:

registered_users_conv = registered_users_conv.pivot(columns='event', values='users', index=None)

In [52]:
registered_users_conv

event,app_start,choose_item,purchase,search,tap_basket
0,40991,37926,24880,40482,34517


<b>Теперь для каждого шага считаем процент пользователей, перешедших в следующий шаг</b>

In [58]:
registered_users_conv['search/choose_item'] = (registered_users_conv['choose_item'] / \
                                               registered_users_conv['search']).mul(100).round(2)

In [59]:
registered_users_conv['choose_item/tap_basket'] = (registered_users_conv['tap_basket'] / \
                                                   registered_users_conv['choose_item']).mul(100).round(2)

In [60]:
registered_users_conv['tap_basket/purchase'] = (registered_users_conv['purchase'] / \
                                                registered_users_conv['tap_basket']).mul(100).round(2)

In [61]:
registered_users_conv

event,app_start,choose_item,purchase,search,tap_basket,search/choose_item,choose_item/tap_basket,tap_basket/purchase
0,40991,37926,24880,40482,34517,93.69,91.01,72.08


<b>Вывод: Наибольшее кол-во зарегистрированных пользователей отваливается на этапе пеехода из корзины в совершение покупки</b>

<hr style="border:1px solid">

### 7.3 Теперь считаем относительную конверсию для незарегистрированных пользователей

In [25]:
# Фильтруем df по условию регистрации пользователя

not_registered_users = full_data.query('is_registered != "registered"')

In [26]:
not_registered_users.head()

,date_actions,event,purchase_sum,device_id,date_register,is_registered
0,2020-01-01,app_start,NaN,669460,2020-01-07,not_registered
1,2020-01-01,app_start,NaN,833621,2020-01-01,not_registered
2,2020-01-01,app_start,NaN,1579237,2020-01-01,not_registered
3,2020-01-01,app_start,NaN,1737182,2020-01-01,not_registered
4,2020-01-01,app_start,NaN,4029024,NaN,not_registered


In [29]:
# Группируем df по событию и считаем кол-во пользователей в каждом шаге

not_registered_users_steps = not_registered_users.groupby('event') \
.agg({'device_id':'nunique'}) \
.rename(columns={'device_id':'users'}) \
.reset_index()

In [30]:
not_registered_users_steps

,event,users
0,app_install,154597
1,app_start,190884
2,choose_item,155691
3,purchase,67753
4,search,184488
5,tap_basket,125414


In [31]:
# Для удобства форматируем df с помощью функции pivot

not_registered_users_steps.index = [0] * len(not_registered_users_steps)

In [32]:
not_registered_users_pivoting = not_registered_users_steps.pivot(index=None, columns='event', values='users')

In [33]:
not_registered_users_pivoting

event,app_install,app_start,choose_item,purchase,search,tap_basket
0,154597,190884,155691,67753,184488,125414


<b>Теперь для каждого шага считаем процент пользователей, перешедших в следующий шаг</b>

In [76]:
not_registered_users_pivoting['search/choose_item'] = (not_registered_users_pivoting['choose_item'] / not_registered_users_pivoting['search']).mul(100).round(2)

In [77]:
not_registered_users_pivoting['choose_item/tap_basket'] = (not_registered_users_pivoting['tap_basket'] / not_registered_users_pivoting['choose_item']).mul(100).round(2)

In [78]:
not_registered_users_pivoting['tap_basket/purchase'] = (not_registered_users_pivoting['purchase'] / not_registered_users_pivoting['tap_basket']).mul(100).round(2)

In [79]:
not_registered_users_pivoting

event,app_install,app_start,choose_item,purchase,search,tap_basket,search/choose_item,choose_item/tap_basket,tap_basket/purchase
0,154597,190884,155691,67753,184488,125414,84.39,80.55,54.02


<b>Вывод: Наибольшее кол-во незарегистрированных пользователей отваливается на этапе пеехода из корзины в совершение покупки</b>

<hr style="border:1px solid">

## 8. Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку

<b>Конверсией в первую покупку будем считать кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение </b>

In [37]:
# Создаем df

conversion_purchase = df.drop(['os_name','gender','city','purchase_sum'], axis=1).query('event == "app_start" or event == "purchase"').copy() 

In [38]:
# Убираем из данных тех пользователей, которые пришли с не определенных каналов

conversion_purchase = conversion_purchase.loc[~(conversion_purchase['utm_source'] == "-")]

In [39]:
conversion_purchase.head()

,date,event,device_id,utm_source
1,2020-01-01,app_start,833621,vk_ads
2,2020-01-01,app_start,1579237,referal
3,2020-01-01,app_start,1737182,facebook_ads
4,2020-01-01,app_start,4029024,facebook_ads
6,2020-01-01,app_start,8383386,vk_ads


In [40]:
# Для вычисления для каждого пользователя его первую дату открытия приложения создаем отдельный столбец

conversion_purchase['first_start'] = conversion_purchase.query('event == "app_start"').groupby('device_id')['date'].transform('min')

In [41]:
conversion_purchase

,date,event,device_id,utm_source,first_start
1,2020-01-01,app_start,833621,vk_ads,2020-01-01
2,2020-01-01,app_start,1579237,referal,2020-01-01
3,2020-01-01,app_start,1737182,facebook_ads,2020-01-01
4,2020-01-01,app_start,4029024,facebook_ads,2020-01-01
6,2020-01-01,app_start,8383386,vk_ads,2020-01-01
...,...,...,...,...,...
2507467,2020-03-31,purchase,13278240,instagram_ads,NaN
2507469,2020-03-31,purchase,11864116,google_ads,NaN
2507470,2020-03-31,purchase,3403969,google_ads,NaN
2507471,2020-03-31,purchase,26120362,google_ads,NaN


In [42]:
# Для вычисления для каждого пользователя его первую дату покупки создаем отдельный столбец

conversion_purchase['first_purchase'] = conversion_purchase.query('event =="purchase"').groupby('device_id')['date'].transform('min')

In [43]:
conversion_purchase

,date,event,device_id,utm_source,first_start,first_purchase
1,2020-01-01,app_start,833621,vk_ads,2020-01-01,NaN
2,2020-01-01,app_start,1579237,referal,2020-01-01,NaN
3,2020-01-01,app_start,1737182,facebook_ads,2020-01-01,NaN
4,2020-01-01,app_start,4029024,facebook_ads,2020-01-01,NaN
6,2020-01-01,app_start,8383386,vk_ads,2020-01-01,NaN
...,...,...,...,...,...,...
2507467,2020-03-31,purchase,13278240,instagram_ads,NaN,2020-03-31
2507469,2020-03-31,purchase,11864116,google_ads,NaN,2020-03-31
2507470,2020-03-31,purchase,3403969,google_ads,NaN,2020-02-12
2507471,2020-03-31,purchase,26120362,google_ads,NaN,2020-03-31


In [44]:
# Фильтруем df по условию даты первого открытия приложения и даты первой покупки

conversion_purchase = conversion_purchase.query('(event == "app_start" and date == first_start) or (event == "purchase" and date == first_purchase)').copy()

In [45]:
conversion_purchase

,date,event,device_id,utm_source,first_start,first_purchase
1,2020-01-01,app_start,833621,vk_ads,2020-01-01,NaN
2,2020-01-01,app_start,1579237,referal,2020-01-01,NaN
3,2020-01-01,app_start,1737182,facebook_ads,2020-01-01,NaN
4,2020-01-01,app_start,4029024,facebook_ads,2020-01-01,NaN
6,2020-01-01,app_start,8383386,vk_ads,2020-01-01,NaN
...,...,...,...,...,...,...
2507462,2020-03-31,purchase,17964402,instagram_ads,NaN,2020-03-31
2507463,2020-03-31,purchase,8700073,vk_ads,NaN,2020-03-31
2507467,2020-03-31,purchase,13278240,instagram_ads,NaN,2020-03-31
2507469,2020-03-31,purchase,11864116,google_ads,NaN,2020-03-31


In [46]:
# Заменяем пропущенные значения на 0

conversion_purchase.fillna(0, inplace=True)

In [47]:
conversion_purchase

,date,event,device_id,utm_source,first_start,first_purchase
1,2020-01-01,app_start,833621,vk_ads,2020-01-01,0
2,2020-01-01,app_start,1579237,referal,2020-01-01,0
3,2020-01-01,app_start,1737182,facebook_ads,2020-01-01,0
4,2020-01-01,app_start,4029024,facebook_ads,2020-01-01,0
6,2020-01-01,app_start,8383386,vk_ads,2020-01-01,0
...,...,...,...,...,...,...
2507462,2020-03-31,purchase,17964402,instagram_ads,0,2020-03-31
2507463,2020-03-31,purchase,8700073,vk_ads,0,2020-03-31
2507467,2020-03-31,purchase,13278240,instagram_ads,0,2020-03-31
2507469,2020-03-31,purchase,11864116,google_ads,0,2020-03-31


<b>Чтобы в дальнейшем считать конверсию, если нет пропуска в столбцах первой покупки или открытия приложения, поставим 1, если есть - 0</b>

In [48]:
conversion_purchase['first_start'] = np.where(conversion_purchase['first_start'] == 0, 0,1)

In [49]:
conversion_purchase['first_purchase'] = np.where(conversion_purchase['first_purchase'] == 0, 0,1)

In [50]:
conversion_purchase

,date,event,device_id,utm_source,first_start,first_purchase
1,2020-01-01,app_start,833621,vk_ads,1,0
2,2020-01-01,app_start,1579237,referal,1,0
3,2020-01-01,app_start,1737182,facebook_ads,1,0
4,2020-01-01,app_start,4029024,facebook_ads,1,0
6,2020-01-01,app_start,8383386,vk_ads,1,0
...,...,...,...,...,...,...
2507462,2020-03-31,purchase,17964402,instagram_ads,0,1
2507463,2020-03-31,purchase,8700073,vk_ads,0,1
2507467,2020-03-31,purchase,13278240,instagram_ads,0,1
2507469,2020-03-31,purchase,11864116,google_ads,0,1


In [51]:
# Группируем по каналам и считаем кол-во пользователей

grouped_data = conversion_purchase.groupby('utm_source') \
.agg({'first_start':'sum','first_purchase':'sum'})

In [52]:
grouped_data

,first_start,first_purchase
utm_source,,
facebook_ads,20429,8140
google_ads,33110,10483
instagram_ads,26370,10036
referal,12908,5937
vk_ads,29378,11682
yandex-direct,36088,11225


In [55]:
grouped_data['conversion'] = (grouped_data.first_purchase / grouped_data.first_start).round(2)

In [56]:
grouped_data.sort_values('conversion')

,first_start,first_purchase,conversion
utm_source,,,
yandex-direct,36088,11225,0.31
google_ads,33110,10483,0.32
instagram_ads,26370,10036,0.38
facebook_ads,20429,8140,0.40
vk_ads,29378,11682,0.40
referal,12908,5937,0.46


<b>Видим, что самую низкую конверсию в первую покупку показали пользователи, которые пришли с Яндекса.</b>

<hr style="border:1px solid">

## 9. Пользователи, пришедшие с какого канала, имеют медианный первый чек выше?

In [57]:
# Создаем новый df, убираем ненужные столбцы

check = df.drop(['os_name','gender','city'], axis=1).copy()

In [58]:
# Убираем из данных пользователей, пришедших с не определенного канала и оставляем только событие покупки

check = check.loc[~(check['utm_source'] == "-")].query('event == "purchase"')

In [60]:
check.head()

,date,event,purchase_sum,device_id,utm_source
7692,2020-01-01,purchase,374.0,77448,yandex-direct
7693,2020-01-01,purchase,98.5,7458607,instagram_ads
7694,2020-01-01,purchase,304.5,3520947,instagram_ads
7695,2020-01-01,purchase,153.5,1170605,google_ads
7697,2020-01-01,purchase,580.5,27612889,google_ads


In [61]:
# Создаем столбец с первой покупкой пользователей

check['first_purchase'] = check.groupby('device_id')['date'].transform('min')

In [63]:
check.head()

,date,event,purchase_sum,device_id,utm_source,first_purchase
7692,2020-01-01,purchase,374.0,77448,yandex-direct,2020-01-01
7693,2020-01-01,purchase,98.5,7458607,instagram_ads,2020-01-01
7694,2020-01-01,purchase,304.5,3520947,instagram_ads,2020-01-01
7695,2020-01-01,purchase,153.5,1170605,google_ads,2020-01-01
7697,2020-01-01,purchase,580.5,27612889,google_ads,2020-01-01


In [65]:
# Фильтруем данные - оставляем только первые покупки

check = check.query('date == first_purchase')

In [66]:
check.head()

,date,event,purchase_sum,device_id,utm_source,first_purchase
7692,2020-01-01,purchase,374.0,77448,yandex-direct,2020-01-01
7693,2020-01-01,purchase,98.5,7458607,instagram_ads,2020-01-01
7694,2020-01-01,purchase,304.5,3520947,instagram_ads,2020-01-01
7695,2020-01-01,purchase,153.5,1170605,google_ads,2020-01-01
7697,2020-01-01,purchase,580.5,27612889,google_ads,2020-01-01


In [ ]:
# Считаем медианные чеки по каналам

median_check = check.groupby('utm_source') \
.agg({'purchase_sum':'median'}) \
.sort_values('purchase_sum', ascending=False)

In [307]:
median_check

,purchase_sum
utm_source,
referal,395.50
instagram_ads,393.50
yandex-direct,393.00
vk_ads,392.50
google_ads,389.50
facebook_ads,389.25


<b>Видим, что пользователи, пришедшие с реферальной программы, имеют медианный чек выше</b>

<hr style="border:1px solid">

## 10. Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI

<b>
Данные по затратам на рекламу:
<br><br>
Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.
</b>

In [ ]:
# Создаем переменную с затаратами на рекламу

expenditures = [8590498, 10534878, 8561626,  9553531, 10491707]

In [402]:
# Создаем новый df, убирая ненужные столбцы из основного df

romi = df.drop(columns= ['os_name', 'city','gender'])

In [403]:
# Фильтурем df - убираем не определенные каналы и оставляем в событиях покупки пользователей - доходы от маркетинга

romi = romi.query('utm_source != "-" and event == "purchase"')

In [404]:
romi

,date,event,purchase_sum,device_id,utm_source
7692,2020-01-01,purchase,374.0,77448,yandex-direct
7693,2020-01-01,purchase,98.5,7458607,instagram_ads
7694,2020-01-01,purchase,304.5,3520947,instagram_ads
7695,2020-01-01,purchase,153.5,1170605,google_ads
7697,2020-01-01,purchase,580.5,27612889,google_ads
...,...,...,...,...,...
2507467,2020-03-31,purchase,6948.0,13278240,instagram_ads
2507469,2020-03-31,purchase,6546.5,11864116,google_ads
2507470,2020-03-31,purchase,3495.0,3403969,google_ads
2507471,2020-03-31,purchase,2857.5,26120362,google_ads


In [406]:
# Группируем данные по каналу и считаем прибыль от каждого

grouped_romi = romi.groupby('utm_source', as_index=False) \
.agg({'purchase_sum':'sum'}) \
.rename(columns={'purchase_sum':'profit'})

In [407]:
grouped_romi

,utm_source,profit
0,facebook_ads,12249901.0
1,google_ads,12868276.0
2,instagram_ads,14546969.0
3,referal,8837044.5
4,vk_ads,16389652.5
5,yandex-direct,13915368.0


In [408]:
# Удаляем канал referal

grouped_romi.drop(3, inplace=True)

In [409]:
grouped_romi

,utm_source,profit
0,facebook_ads,12249901.0
1,google_ads,12868276.0
2,instagram_ads,14546969.0
4,vk_ads,16389652.5
5,yandex-direct,13915368.0


In [410]:
grouped_romi = grouped_romi.reset_index( drop = True)

In [411]:
grouped_romi

,utm_source,profit
0,facebook_ads,12249901.0
1,google_ads,12868276.0
2,instagram_ads,14546969.0
3,vk_ads,16389652.5
4,yandex-direct,13915368.0


In [412]:
# Создаем столбец с маркетинговыми расходами по каждому каналу

grouped_romi['expenditures'] = pd.Series(expenditures)

In [413]:
grouped_romi

,utm_source,profit,expenditures
0,facebook_ads,12249901.0,8590498
1,google_ads,12868276.0,10534878
2,instagram_ads,14546969.0,8561626
3,vk_ads,16389652.5,9553531
4,yandex-direct,13915368.0,10491707


In [362]:
grouped_romi['expenditures'] = pd.Series(expenditures)

In [414]:
grouped_romi

,utm_source,profit,expenditures
0,facebook_ads,12249901.0,8590498
1,google_ads,12868276.0,10534878
2,instagram_ads,14546969.0,8561626
3,vk_ads,16389652.5,9553531
4,yandex-direct,13915368.0,10491707


In [415]:
# Расчитываем ROMI 

grouped_romi['romi'] = (grouped_romi['profit'] - grouped_romi['expenditures']) / grouped_romi['expenditures']

In [420]:
grouped_romi['romi'] = grouped_romi['romi'].round(2)

In [421]:
grouped_romi

,utm_source,profit,expenditures,romi
0,facebook_ads,12249901.0,8590498,0.43
1,google_ads,12868276.0,10534878,0.22
2,instagram_ads,14546969.0,8561626,0.70
3,vk_ads,16389652.5,9553531,0.72
4,yandex-direct,13915368.0,10491707,0.33


<b>Самый высокий ROMI имеет ВК.</b>